# FINN - How to work with ONNX

<font size="3">This notebook should give an overview of ONNX ProtoBuf and help to create and manipulate an ONNX model and use FINN functions to work with it. There may be overlaps to other notebooks, like [FINN-ModelWrapper](FINN-ModelWrapper.ipynb) and [FINN-CustomOps](FINN-CustomOps.ipynb), but this notebook should give an overview about the handling of ONNX models in FINN. </font>

## Outline
* #### How to create a simple model

### How to create a simple model

<font size="3">To explain how to create an ONNX graph a simple example with mathematical operations is used. All nodes are from the [standard operations library of ONNX](https://github.com/onnx/onnx/blob/master/docs/Operators.md).

First ONNX is imported, then the helper function can be used to make a node.</font>

In [1]:
import onnx

Add1_node = onnx.helper.make_node(
    'Add',
    inputs=['in1', 'in2'],
    outputs=['sum1'],
)

<font size="3">The first attribute of the node is the operation type. In this case it is `'Add'`, so it is an adder node. Then the input names are passed to the node and at the end a name is assigned to the output.
    
For this example we want two other adder nodes, one abs node and the output shall be rounded so one round node is needed. </font>

In [2]:
Add2_node = onnx.helper.make_node(
    'Add',
    inputs=['sum1', 'in3'],
    outputs=['sum2'],
)

Add3_node = onnx.helper.make_node(
    'Add',
    inputs=['sum2', 'abs1'],
    outputs=['sum3'],
)

Abs_node = onnx.helper.make_node(
    'Abs',
    inputs=['sum2'],
    outputs=['abs1'],
)

Round_node = onnx.helper.make_node(
    'Round',
    inputs=['sum3'],
    outputs=['out1'],
)


<font size="3">Seeing the names of the inputs and outputs of the nodes the structure of the resulting graph can already be recognized. In order to integrate the nodes into a graph environment, the inputs and outputs of the graph have to be specified first. In ONNX all data edges are processed as tensors. So with the helper function tensor value infos are created for the input and output tensors of the graph. For the data type float from ONNX is used. </font>

In [3]:
in1 = onnx.helper.make_tensor_value_info("in1", onnx.TensorProto.FLOAT, [4, 4])
in2 = onnx.helper.make_tensor_value_info("in2", onnx.TensorProto.FLOAT, [4, 4])
in3 = onnx.helper.make_tensor_value_info("in3", onnx.TensorProto.FLOAT, [4, 4])
out1 = onnx.helper.make_tensor_value_info("out1", onnx.TensorProto.FLOAT, [4, 4])

<font size="3">Now the graph can be built. First all nodes are passed. Here it is to be noted that it requires a certain sequence. The nodes must be instantiated in their dependencies to each other. This means Add2 must not be listed before Add1, because Add2 depends on the result of Add1. A name is then assigned to the graph. This is followed by the inputs and outputs. 

`value_info` of the graph contains the remaining tensors within the graph. When creating the nodes we have already defined names for the inner edges and now these are assigned tensors of the datatype float and a certain shape.</font>

In [4]:
 graph = onnx.helper.make_graph(
        nodes=[
            Add1_node,
            Add2_node,
            Abs_node,
            Add3_node,
            Round_node,
        ],
        name="simple_graph",
        inputs=[in1, in2, in3],
        outputs=[out1],
        value_info=[
            onnx.helper.make_tensor_value_info("sum1", onnx.TensorProto.FLOAT, [4, 4]),
            onnx.helper.make_tensor_value_info("sum2", onnx.TensorProto.FLOAT, [4, 4]),
            onnx.helper.make_tensor_value_info("abs1", onnx.TensorProto.FLOAT, [4, 4]),
            onnx.helper.make_tensor_value_info("sum3", onnx.TensorProto.FLOAT, [4, 4]),
        ],
    )


<font size="3">**Important**: In our example, the shape of the tensors does not change during the calculation. This is not always the case. So you have to make sure that you specify the shape correctly.

Now a model can be created from the graph and saved using the `.save` function. The model is saved in .onnx format and can be reloaded with `onnx.load()`. This also means that you can easily share your own model in .onnx format with others.</font>

In [5]:
onnx_model = onnx.helper.make_model(graph, producer_name="simple-model")
onnx.save(onnx_model, 'simple_model.onnx')

<font size='3'>To visualize the created model, [netron](https://github.com/lutzroeder/netron) can be used. Netron is a visualizer for neural network, deep learning and machine learning models. <font>

In [6]:
import netron
netron.start('simple_model.onnx', port=8081, host="0.0.0.0")

Serving 'simple_model.onnx' at http://0.0.0.0:8081


In [7]:
%%html
<iframe src="http://0.0.0.0:8081/" style="position: relative; width: 100%;" height="400"></iframe>

<font size="3">Netron also allows you to interactively explore the model. If you click on a node, the node attributes will be displayed. 

In order to test the resulting model, a function is first written in Python that calculates the expected output. Because numpy arrays are to be used, numpy is imported first.</font>

In [8]:
import numpy as np

def expected_output(in1, in2, in3):
    sum1 = np.add(in1, in2)
    sum2 = np.add(sum1, in3)
    abs1 = np.absolute(sum2)
    sum3 = np.add(abs1, sum2)
    return np.round(sum3)

<font size="3">Then the values for the three inputs are calculated. Random numbers are used.</font>

In [9]:
in1_values =np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)
in2_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)
in3_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)

<font size="3">We can easily pass the values to the function we just wrote to calculate the expected result. For the created model the inputs must be summarized in a dictionary, which is then passed on to the model.</font>

In [13]:
input_dict = {}
input_dict["in1"] = in1_values
input_dict["in2"] = in2_values
input_dict["in3"] = in3_values

<font size="3">To run the model and calculate the output, [onnxruntime](https://github.com/microsoft/onnxruntime) can be used. ONNX Runtime is a performance-focused complete scoring engine for Open Neural Network Exchange (ONNX) models. The `.InferenceSession` function can be used to create a session of the model and .run can be used to execute the model. </font>

In [21]:
import onnxruntime as rt

sess = rt.InferenceSession(onnx_model.SerializeToString())
output = sess.run(None, input_dict)

<font size="3">The input values are also transferred to the reference function. Now the output of the execution of the model can be compared with that of the reference. </font>

In [26]:
ref_output= expected_output(in1_values, in2_values, in3_values)
assert output[0].all() == ref_output.all()